In [1]:
import os
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
import gc
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
DATA_DIRECTORY = ""

In [3]:
train = pd.read_csv(os.path.join(DATA_DIRECTORY, 'train.csv'))
test = pd.read_csv(os.path.join(DATA_DIRECTORY, 'test.csv'))
labels = pd.read_csv(os.path.join(DATA_DIRECTORY, 'labels.csv'))

In [4]:
labels = labels.to_numpy()
test_id = test['SK_ID_CURR']
train_id = train['SK_ID_CURR']

In [5]:
train = train.drop(['SK_ID_CURR'], axis=1)
test = test.drop(['SK_ID_CURR'], axis=1)

In [6]:
imputer = SimpleImputer(strategy = 'median')
imputer.fit(train)
train = imputer.transform(train)
test = imputer.transform(test)

In [7]:
scaler = MinMaxScaler(feature_range = (0, 1))
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

In [8]:
def model(features, test_features, labels, test_ids, n_folds = 5):
        
    print('Training Data Shape: ', features.shape)
    print('Testing Data Shape: ', test_features.shape)
    
    # Create the kfold object
    k_fold = KFold(n_splits = n_folds, shuffle = True, random_state = 8888)
    
    # Empty array for test predictions
    test_predictions = np.zeros(test_features.shape[0])
    
    # Empty array for out of fold validation predictions
    out_of_fold = np.zeros(features.shape[0])
    
    # Lists for recording validation and training scores
    valid_scores = []
    train_scores = []

    ratio = (labels == 0).sum()/ (labels == 1).sum()
    
    # Iterate through each fold
    for train_indices, valid_indices in k_fold.split(features):
        
        # Training data for the fold
        train_features, train_labels = features[train_indices], labels[train_indices]
        # Validation data for the fold
        valid_features, valid_labels = features[valid_indices], labels[valid_indices]
        
        # Create the model
        model = XGBClassifier(
                  n_estimators=5000,
                  learning_rate=0.01,
                  max_depth=11,
                  objective='binary:logistic', 
                  gamma=0.098, 
                  subsample=0.708,
                  reg_alpha=3.564, 
                  reg_lambda=4.930,
                  random_state=8888,
                  seed=88888,
                  scale_pos_weight=ratio,
                  colsample_bytree= 0.613,
                  min_child_weight= 6,
                  tree_method='gpu_hist',
                  predictor='gpu_predictor'
                  )
        
        # Train the model
        model.fit(train_features, train_labels, eval_metric = 'auc',
                  eval_set = [(train_features, train_labels), (valid_features, valid_labels)],
                  early_stopping_rounds = 2500, verbose = 1000)
        
        # Make predictions
        test_predictions += model.predict_proba(test_features)[:, 1] / k_fold.n_splits
        
        # Record the out of fold predictions
        out_of_fold[valid_indices] = model.predict_proba(valid_features)[:, 1]
        
        evals_result = model.evals_result()
        valid_score = max(evals_result['validation_1']['auc'])
        train_score = max(evals_result['validation_0']['auc'])
        
        valid_scores.append(valid_score)
        train_scores.append(train_score)
        
        # Clean up memory
        gc.enable()
        del model, train_features, valid_features
        gc.collect()
        
    # Make the submission dataframe
    submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': test_predictions})
    
    # Overall validation score
    valid_auc = roc_auc_score(labels, out_of_fold)
    
    # Add the overall scores to the metrics
    valid_scores.append(valid_auc)
    train_scores.append(np.mean(train_scores))
    
    # Needed for creating dataframe of validation scores
    fold_names = list(range(n_folds))
    fold_names.append('overall')
    
    # Dataframe of validation scores
    metrics = pd.DataFrame({'fold': fold_names,
                            'train': train_scores,
                            'valid': valid_scores}) 
    
    return submission, metrics

In [9]:
submission, metrics = model(train, train, labels, train_id)

Training Data Shape:  (307506, 649)
Testing Data Shape:  (307506, 649)


C:\Users\sakib\anaconda3\envs\CSE499\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\sakib\anaconda3\envs\CSE499\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[0]	validation_0-auc:0.79603	validation_1-auc:0.69395
[1000]	validation_0-auc:0.99274	validation_1-auc:0.79005
[2000]	validation_0-auc:0.99952	validation_1-auc:0.78887
[3000]	validation_0-auc:0.99997	validation_1-auc:0.78836
[3359]	validation_0-auc:0.99999	validation_1-auc:0.78848
[0]	validation_0-auc:0.79744	validation_1-auc:0.68108
[1000]	validation_0-auc:0.99308	validation_1-auc:0.78418
[2000]	validation_0-auc:0.99957	validation_1-auc:0.78365
[3000]	validation_0-auc:0.99997	validation_1-auc:0.78333
[3651]	validation_0-auc:0.99999	validation_1-auc:0.78344
[0]	validation_0-auc:0.79517	validation_1-auc:0.68705
[1000]	validation_0-auc:0.99313	validation_1-auc:0.78805
[2000]	validation_0-auc:0.99960	validation_1-auc:0.78692
[3000]	validation_0-auc:0.99998	validation_1-auc:0.78678
[3582]	validation_0-auc:1.00000	validation_1-auc:0.78690
[0]	validation_0-auc:0.79514	validation_1-auc:0.68986
[1000]	validation_0-auc:0.99341	validation_1-auc:0.78994
[2000]	validation_0-auc:0.99960	validation_

In [10]:
submission = submission['TARGET'].to_numpy()

In [11]:
from sklearn.metrics import roc_curve

In [12]:
fpr, tpr, thresholds = roc_curve(labels, submission)
# Calculate the G-mean
gmean = np.sqrt(tpr * (1 - fpr))

# Find the optimal threshold
index = np.argmax(gmean)
bestThreshold = thresholds[index]

In [13]:
target = np.where(submission > bestThreshold, 1, 0)

In [14]:
del fpr, gmean, imputer, index, DATA_DIRECTORY, thresholds, tpr, submission, metrics, bestThreshold
gc.collect()

558

In [15]:
from pytorch_tabnet.tab_model import TabNetClassifier
import torch

In [16]:
def model(features, test_features, labels, test_ids, n_folds = 5):
        
    print('Training Data Shape: ', features.shape)
    print('Testing Data Shape: ', test_features.shape)
    
    # Create the kfold object
    k_fold = KFold(n_splits = n_folds, shuffle = True, random_state = 88)
    
    # Empty array for test predictions
    test_predictions = np.zeros(test_features.shape[0])
    
    # Empty array for out of fold validation predictions
    out_of_fold = np.zeros(features.shape[0])
    
    # Lists for recording validation and training scores
    valid_scores = []
    train_scores = []

    # Iterate through each fold
    for train_indices, valid_indices in k_fold.split(features):
        
        # Training data for the fold
        train_features, train_labels = features[train_indices], labels[train_indices]
        # Validation data for the fold
        valid_features, valid_labels = features[valid_indices], labels[valid_indices]

        # Create the model
        model = TabNetClassifier(
                    n_d=32, 
                    n_a=32, 
                    n_steps=10,
                    gamma=0.098, 
                    n_independent=2, 
                    n_shared=2,
                    lambda_sparse=1e-3, 
                    momentum=0.4, 
                    clip_value=2.,
                    optimizer_fn=torch.optim.Adam,
                    scheduler_params = {"gamma": 0.95,
                                    "step_size": 20},
                    optimizer_params=dict(lr=2e-2),
                    scheduler_fn=torch.optim.lr_scheduler.StepLR, 
                    epsilon=1e-15, verbose = 0,
                    device_name='cuda'
                )
        
        # Train the model
        model.fit(
            train_features, train_labels,
            eval_set=[(train_features, train_labels), (valid_features, valid_labels)],  
            eval_name=['train', 'valid'],
            eval_metric=['auc'],
            max_epochs=1000 , patience=50,
            batch_size=1024, virtual_batch_size=128,
            num_workers=0,
            weights=1,
            drop_last=False
        )

        print(model)
        
        # Make predictions
        test_predictions += model.predict_proba(test_features)[:, 1] / k_fold.n_splits
        
        # Record the out of fold predictions
        out_of_fold[valid_indices] = model.predict_proba(valid_features)[:, 1]

        # Record the best score
        valid_score = roc_auc_score(valid_labels, model.predict(valid_features))
        train_score = roc_auc_score(train_labels, model.predict(train_features))
        
        valid_scores.append(valid_score)
        train_scores.append(train_score)
        
        # Clean up memory
        gc.enable()
        del model, train_features, valid_features
        gc.collect()
        
    # Make the submission dataframe
    submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': test_predictions})
    
    # Overall validation score
    valid_auc = roc_auc_score(labels, out_of_fold)
    
    # Add the overall scores to the metrics
    valid_scores.append(valid_auc)
    train_scores.append(np.mean(train_scores))
    
    # Needed for creating dataframe of validation scores
    fold_names = list(range(n_folds))
    fold_names.append('overall')
    
    # Dataframe of validation scores
    metrics = pd.DataFrame({'fold': fold_names,
                            'train': train_scores,
                            'valid': valid_scores}) 
    
    return submission, metrics

In [17]:
submission, metrics = model(train, train, target, train_id)

Training Data Shape:  (307506, 649)
Testing Data Shape:  (307506, 649)

Early stopping occurred at epoch 52 with best_epoch = 2 and best_valid_auc = 0.90614
Best weights from best epoch are automatically used!
TabNetClassifier(n_d=32, n_a=32, n_steps=10, gamma=0.098, cat_idxs=[], cat_dims=[], cat_emb_dim=1, n_independent=2, n_shared=2, epsilon=1e-15, momentum=0.4, lambda_sparse=0.001, seed=0, clip_value=2.0, verbose=0, optimizer_fn=<class 'torch.optim.adam.Adam'>, optimizer_params={'lr': 0.02}, scheduler_fn=<class 'torch.optim.lr_scheduler.StepLR'>, scheduler_params={'gamma': 0.95, 'step_size': 20}, mask_type='sparsemax', input_dim=649, output_dim=2, device_name='cuda')

Early stopping occurred at epoch 54 with best_epoch = 4 and best_valid_auc = 0.899
Best weights from best epoch are automatically used!
TabNetClassifier(n_d=32, n_a=32, n_steps=10, gamma=0.098, cat_idxs=[], cat_dims=[], cat_emb_dim=1, n_independent=2, n_shared=2, epsilon=1e-15, momentum=0.4, lambda_sparse=0.001, seed=0

In [18]:
print('TabNet metrics')
print(metrics)

TabNet metrics
      fold     train     valid
0        0  0.839724  0.826682
1        1  0.865540  0.818954
2        2  0.836900  0.824502
3        3  0.850394  0.821780
4        4  0.850088  0.825996
5  overall  0.848529  0.901817


In [19]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
tn = submission['TARGET'].to_numpy().reshape(-1, 1)
scaler.fit(tn)
tg = scaler.transform(tn)

In [20]:
submission = pd.DataFrame({'SK_ID_CURR': test_id, 'TARGET': tg[:,0]})

ValueError: array length 307506 does not match index length 48744

In [ ]:
submission.to_csv('xgboost-tabnet.csv', index = False)

In [ ]:
pred = submission['TARGET'].to_numpy()
import matplotlib.pyplot as plt

fpr, tpr, _ = roc_curve(labels[:,0],  pred)
auc = roc_auc_score(labels[:,0], pred)
plt.plot(fpr,tpr,label="XGBoost-TabNet")
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')  
plt.legend(loc=4)
plt.show()